In [225]:
#OBS pip install thefuzz
import pandas as pd
from thefuzz import process
from thefuzz import fuzz, process
# Load the Excel file into a DataFrame
file_path_excel = 'karakterer.xlsx'  # Ret sti
df_karakter = pd.read_excel(file_path_excel, sheet_name='Ark2')

df_karak_kom = pd.read_excel(file_path_excel, sheet_name='Ark3')
df_karak_kom = df_karak_kom.rename(columns={
    'Beliggenhedskommune': 'belkom',
    'Gennemsnit - Bundne prøver': 'Average_Grade'
})


# Load the CSV file into a DataFrame bolig_laeger_st_DST
csv_file_path = 'outputdata/bolig_laeger_st_DST.csv'   # Ret sti
df_csv = pd.read_csv(csv_file_path)

df_csv['Municipality_cleaned'] = df_csv['Municipiality'].str.replace('Kommune', '', regex=False).str.strip()

# Get unique values for Beliggenhedskommune in df_karakter
beliggenhedskommune_options = df_karakter['Beliggenhedskommune'].unique()

# Function to match Municipality_cleaned with Beliggenhedskommune using fuzzy matching
def match_municipality(muni, options, threshold=90):
    match, score = process.extractOne(muni, options)
    return match if score >= threshold else None

# Apply fuzzy matching to each Municipality_cleaned in df_csv
df_csv['Matched_Kommune'] = df_csv['Municipality_cleaned'].apply(match_municipality, options=beliggenhedskommune_options)
# Perform the merge with the matched municipality
merged_df = pd.merge(df_csv, 
                     df_karakter[['Afdeling', 'Karakter', 'Soc_ref', '1 årig - Socioøkonomisk Reference Forskel', 'Beliggenhedskommune']], 
                     left_on=['Schoolname', 'Matched_Kommune'], 
                     right_on=['Afdeling', 'Beliggenhedskommune'], 
                     how='left')


#Second merge with average grades
final_merged_df = pd.merge(merged_df,
                           df_karak_kom[['belkom', 'Average_Grade', 'Indikatorsvar']],
                           left_on='Matched_Kommune',
                           right_on='belkom',
                           how='left')

#Dropping variables
final_merged_df.drop(['Municipality_cleaned', 'Beliggenhedskommune', 'belkom', 'Municipality_cleaned', 'Soc_ref'  ], axis=1, inplace=True)


#Data cleaning
import numpy as np

# Fill NaN in 'Karakter' with 'Average_Grade' and create a dummy for imputed values
final_merged_df['Karakter_imputed'] = np.where(final_merged_df['Karakter'].isna(), 1, 0)
final_merged_df['Karakter'] = final_merged_df['Karakter'].fillna(final_merged_df['Average_Grade'])

# Fill NaN in 'Trivsel' with 'Indikator svar' and create a dummy for imputed values
final_merged_df['indikator_imputed'] = np.where(final_merged_df['Trivsel'].isna(), 1, 0)
final_merged_df['Trivsel'] = final_merged_df['Trivsel'].fillna(final_merged_df['Indikatorsvar'])

# Fill NaN in '1 årig - Socioøkonomisk Reference Forskel' with 'På Niveau' and create a dummy for imputed values
final_merged_df['Socioeconomic_Ref_Imputed'] = np.where(final_merged_df['1 årig - Socioøkonomisk Reference Forskel'].isna(), 1, 0)
final_merged_df['1 årig - Socioøkonomisk Reference Forskel'] = final_merged_df['1 årig - Socioøkonomisk Reference Forskel'].fillna('På Niveau')

# Dropping of unwanted variabels
NW_merged_df=final_merged_df.drop(['School-grades', 'Matched_Kommune', 'Afdeling'  ], axis=1)

#imputed_columns
imputed_columns = NW_merged_df.loc[:, NW_merged_df.columns.str.lower().str.endswith('_imputed')]

# Sum across rows for columns ending with '_imputed' and create a new column
NW_merged_df['sum_imputed'] = imputed_columns.sum(axis=1)

# Identify columns that end with '_imputed'
sidste = [col for col in NW_merged_df.columns if col.lower().endswith('_imputed')]

# Identify columns that do not end with '_imputed'
forste = [col for col in NW_merged_df.columns if col not in sidste]

# Reorder the DataFrame columns: non-imputed columns first, then imputed columns
NW_merged_df = NW_merged_df[forste + sidste]




,Unnamed: 0,Price,Schoolname,Distance to school,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Karakter,1 årig - Socioøkonomisk Reference Forskel,Average_Grade,Indikatorsvar,Rooms_imputed,Toilets_imputed,Karakter_imputed,indikator_imputed,Socioeconomic_Ref_Imputed,sum_imputed
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,3.502026,Rækkehus,D,Høj,24.60,25.00,...,6.896407,På Niveau,6.896407,3.502026,0.0,0.0,1,1,1,3.0
1,"Pilehusene 84,2600",2995000.0,Herstedøster Skole,0.774,3.700000,Rækkehus,C,Høj,25.60,33.94,...,7.600000,På Niveau,6.774788,3.548950,0.0,0.0,0,0,0,0.0
2,"Magnoliavej 22, st. th.,2600",1795000.0,"Glostrup skole, Søndervang",0.598,3.502026,Ejerlejlighed,D,Høj,24.60,25.00,...,7.200000,På Niveau,6.896407,3.502026,0.0,0.0,0,1,0,1.0
3,"Skolevej 3,2600",4295000.0,Skolerne i Glostrup,0.342,3.502026,Villa,D,Høj,24.60,25.00,...,7.200000,På Niveau,6.896407,3.502026,0.0,0.0,0,1,0,1.0
4,"Falkevej 32,2600",6245000.0,"Glostrup skole, Nordvang",0.279,3.502026,Villa,C,Høj,24.60,25.00,...,6.400000,På Niveau,6.896407,3.502026,0.0,0.0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19846,"Egevej 2,7540",1295000.0,Haderup Skole,7.610,3.700000,Villa,C,Lav,24.90,20.53,...,6.200000,På Niveau,7.177844,3.650689,0.0,0.0,0,0,0,0.0
19847,"Bredgade 66,7540",620000.0,Haderup Skole,7.800,3.700000,Villa,E,Lav,24.90,20.53,...,6.200000,På Niveau,7.177844,3.650689,0.0,0.0,0,0,0,0.0
19848,"Østerhovedvej 3,4942",2695000.0,Østofte Landsbyordning,11.680,3.800000,Villa,A2010,Meget høj,26.30,33.09,...,5.631308,På Niveau,5.631308,3.569307,0.0,0.0,1,0,1,2.0
19849,"Nordre Strandvej 109,3150",17900000.0,Hellebækskolen,1.170,3.800000,Villa,D,Lav,25.52,28.50,...,7.500000,På Niveau,7.314811,3.550135,0.0,0.0,0,0,0,0.0


In [227]:
NW_merged_df.to_csv('outputdata/NW_merged.csv', index=False)

,Unnamed: 0,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,...,Grades_imputed,Toilets_imputed,Muncipiality,Matched_Kommune,Afdeling,Karakter,1 årig - Socioøkonomisk Reference Forskel,Average_Grade,Karakter_imputed,Socioeconomic_Ref_Imputed
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,7.187918,3.728406,Rækkehus,D,Høj,24.60,...,1,0,Glostrup Kommune,Glostrup,NaN,6.896407,På Niveau,6.896407,1,1
1,"Strandpromenaden 21, 1. tv.,3000",6095000.0,Skolen i Bymidten,0.336,7.182988,3.723773,Ejerlejlighed,A2015,Lav,25.52,...,1,0,Helsingør Kommune,Helsingør,Skolen i Bymidten,6.800000,Dårligere end forventet,7.314811,0,0
2,"Hjortdalvej 331,9460",3995000.0,Skovsgård Tranum skole,5.240,6.900000,3.600000,Landejendom,D,Meget lav,25.70,...,0,0,Jammerbugt Kommune,Jammerbugt,Skovsgård Tranum skole,6.000000,På Niveau,6.768619,0,0
3,"Lystgårdsparken 34,8300",2798000.0,Parkvejens Skole,5.160,7.000000,3.700000,Villa,B,Mellem,25.10,...,0,0,Odder Kommune,Odder,Parkvejens Skole,7.300000,På Niveau,7.338372,0,0
4,"Fuglsøvej 92,8420",6000000.0,Molsskolen,3.740,7.400000,3.700000,Villa,D,Høj,25.90,...,0,0,Syddjurs Kommune,Syddjurs,Molsskolen,8.400000,På Niveau,7.160032,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9911,"Langelandsvej 23,6100",1145000.0,Starup-Øsby Skole og Børnehus,1.140,6.400000,3.800000,Villa,D,Høj,26.30,...,0,0,Haderslev Kommune,Haderslev,NaN,6.385789,På Niveau,6.385789,1,1
9912,"Nybovej 7,9640",995000.0,Farsø Skole,0.373,6.900000,3.700000,Villa,D,Mellem,26.30,...,0,0,Vesthimmerlands Kommune,Vesthimmerlands,Farsø Skole,8.200000,Bedre end forventet,7.459388,0,0
9913,"Storskovvej 3,4700",2995000.0,"Fladsåskolen, afd. Korskilde",4.910,7.013961,3.717343,Villa,A2010,Høj,25.00,...,1,0,Næstved Kommune,Næstved,NaN,6.984070,På Niveau,6.984070,1,1
9914,"Gl Landevej 23,9690",725000.0,Fjerritslev Skole,5.620,6.400000,3.600000,Villa,E,Lav,25.70,...,0,0,Jammerbugt Kommune,Jammerbugt,Fjerritslev Skole,6.700000,På Niveau,6.768619,0,0


In [171]:
NW_merged_df=final_merged_df.drop(['School-grades', 'Grades_imputed', 'Matched_Kommune', 'Afdeling'  ], axis=1)
NW_merged_df.head(50)

,Unnamed: 0,Price,Schoolname,Distance to school,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Terasse,Rooms_imputed,Trivsel_imputed,Toilets_imputed,Muncipiality,Karakter,1 årig - Socioøkonomisk Reference Forskel,Average_Grade,Karakter_imputed,Socioeconomic_Ref_Imputed
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,3.728406,Rækkehus,D,Høj,24.60,25.00,...,0,0,1,0,Glostrup Kommune,6.896407,På Niveau,6.896407,1,1
1,"Strandpromenaden 21, 1. tv.,3000",6095000.0,Skolen i Bymidten,0.336,3.723773,Ejerlejlighed,A2015,Lav,25.52,28.50,...,0,0,1,0,Helsingør Kommune,6.800000,Dårligere end forventet,7.314811,0,0
2,"Hjortdalvej 331,9460",3995000.0,Skovsgård Tranum skole,5.240,3.600000,Landejendom,D,Meget lav,25.70,32.85,...,0,0,0,0,Jammerbugt Kommune,6.000000,På Niveau,6.768619,0,0
3,"Lystgårdsparken 34,8300",2798000.0,Parkvejens Skole,5.160,3.700000,Villa,B,Mellem,25.10,30.32,...,0,0,0,0,Odder Kommune,7.300000,På Niveau,7.338372,0,0
4,"Fuglsøvej 92,8420",6000000.0,Molsskolen,3.740,3.700000,Villa,D,Høj,25.90,33.80,...,0,0,0,0,Syddjurs Kommune,8.400000,På Niveau,7.160032,0,0
5,"Korsebjergengen 19,2630",4195000.0,Ole Rømer-Skolen,1.500,3.500000,Villa,C,Høj,24.60,24.40,...,0,0,0,0,Høje-Taastrup Kommune,5.900000,På Niveau,7.003100,0,0
6,"Vestervang 32, 2. tv.,2500",3200000.0,Hanssted Skole,0.283,3.800000,Ejerlejlighed,D,Mellem,23.70,34.00,...,0,0,0,0,Københavns Kommune,8.900000,Bedre end forventet,7.488218,0,0
7,"Capellavej 45,8270",6795000.0,Rundhøjskolen,0.909,3.700000,Villa,C,Høj,24.52,24.58,...,0,0,0,0,Aarhus Kommune,8.100000,På Niveau,7.568960,0,0
8,"Grønkjærsvej 44,7000",2298000.0,"Erritsø Fællesskole, Krogagervej",0.413,3.717343,Villa,D,Høj,25.50,26.00,...,0,0,1,0,Fredericia Kommune,7.200000,På Niveau,6.743536,0,0
9,"Ramsherred 31,5900",795000.0,Ørstedskolen,1.630,3.717343,Villa,B,Høj,26.30,24.57,...,0,0,1,0,Langeland Kommune,6.700000,På Niveau,6.706579,0,0


In [ ]:
#!!!!!!!!!!!!!!!!!! IGNORER ALT HERFRA !!!!!!!!!!!!!!!!!!!!!#

In [165]:
nan_afdeling_df = final_merged_df[final_merged_df['Afdeling'].isna()]
nan_afdeling_df
unique_missing = nan_afdeling_df['Schoolname'].unique()
nan_afdeling_df

,Unnamed: 0,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,...,Grades_imputed,Toilets_imputed,Muncipiality,Matched_Kommune,Afdeling,Karakter,1 årig - Socioøkonomisk Reference Forskel,Average_Grade,Karakter_imputed,Socioeconomic_Ref_Imputed
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,7.187918,3.728406,Rækkehus,D,Høj,24.6,...,1,0,Glostrup Kommune,Glostrup,NaN,6.896407,På Niveau,6.896407,1,1
15,"Steveltvej 17,6100",1845000.0,Starup-Øsby Skole og Børnehus,6.960,6.400000,3.800000,Villa,C,Høj,26.3,...,0,0,Haderslev Kommune,Haderslev,NaN,6.385789,På Niveau,6.385789,1,1
24,"Møllevangsvej 79,8940",1795000.0,Søndermarkskolen,2.960,6.800000,3.800000,Villa,C,Høj,25.8,...,0,0,Randers Kommune,Randers,NaN,6.920515,På Niveau,6.920515,1,1
37,"Rendebæksvej 42,2765",8495000.0,Ung Egedal - Hovedadresse,0.778,7.013961,3.717343,Villa,D,Høj,25.7,...,1,0,Egedal Kommune,Egedal,NaN,7.506144,På Niveau,7.506144,1,1
42,"Stationsvej 20,7330",1695000.0,Dalgasskolen,0.407,7.013961,3.717343,Villa,C,Lav,25.1,...,1,0,Ikast-Brande Kommune,Ikast-Brande,NaN,7.150932,På Niveau,7.150932,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9897,"Råsted Kirkevej 24A,7570",1995000.0,Vemb Skole og Børnehus,5.660,6.100000,3.700000,Villa,E,Lav,25.5,...,0,0,Holstebro Kommune,Holstebro,NaN,7.283131,På Niveau,7.283131,1,1
9905,"Sankt Jørgens Vej 78,5700",2345000.0,Vestre Skole,1.470,5.900000,3.700000,Villa,D,Meget høj,26.3,...,0,0,Svendborg Kommune,Svendborg,NaN,6.892428,På Niveau,6.892428,1,1
9910,"Haldagerlillevej 6,4250",675000.0,Vesterlandsskolen,3.690,7.013961,3.717343,Villa,D,Høj,25.0,...,1,0,Næstved Kommune,Næstved,NaN,6.984070,På Niveau,6.984070,1,1
9911,"Langelandsvej 23,6100",1145000.0,Starup-Øsby Skole og Børnehus,1.140,6.400000,3.800000,Villa,D,Høj,26.3,...,0,0,Haderslev Kommune,Haderslev,NaN,6.385789,På Niveau,6.385789,1,1


In [157]:
filtered_row = nan_afdeling_df[nan_afdeling_df['Schoolname'] == "Østervangsskolen"]
filtered_row

,Unnamed: 0,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,...,Grades_imputed,Toilets_imputed,Muncipiality,Matched_Kommune,Afdeling,Karakter,1 årig - Socioøkonomisk Reference Forskel,Average_Grade,Karakter_imputed,Socioeconomic_Ref_Imputed
3154,"Gl. Hadsundvej 7,8900",3995000.0,Østervangsskolen,0.851,6.6,3.8,Villa,C,Høj,25.8,...,0,0,Randers Kommune,Randers,NaN,6.920515,På Niveau,6.920515,1,1
6154,"Christiansborgvej 9,8930",2775000.0,Østervangsskolen,0.215,6.6,3.8,Villa,D,Høj,25.8,...,0,0,Randers Kommune,Randers,NaN,6.920515,På Niveau,6.920515,1,1
6619,"Østervangsvej 15,8900",4495000.0,Østervangsskolen,0.480,6.6,3.8,Villa,D,Høj,25.8,...,0,0,Randers Kommune,Randers,NaN,6.920515,På Niveau,6.920515,1,1
7066,"Udbyhøjvej 29, 1. tv.,8930",450000.0,Østervangsskolen,0.795,6.6,3.8,Ejerlejlighed,E,Høj,25.8,...,0,0,Randers Kommune,Randers,NaN,6.920515,På Niveau,6.920515,1,1


In [167]:
new_df =nan_afdeling_df[(nan_afdeling_df['Grades_imputed'] == 0) & (nan_afdeling_df['Karakter_imputed'] == 1)]
unik=new_df['Schoolname'].unique()
unik

array(['Starup-Øsby Skole og Børnehus', 'Søndermarkskolen',
       'Sundbyøster Skole', 'Sydmors skole og Børnehus', 'Gerbrandskolen',
       'Tre Ege Skolen', 'Vestre Skole', 'Bjerregrav Skole',
       'Valdemarskolen', 'Glavendrupskolen', 'Humble Skole',
       'Fællesskolen Bevtoft-Over Jerstal, afd. Over Jerstal',
       'Bregnbjergskolen', 'Midtmors Skole', 'Arnborg-Kølkær Skole',
       'Bøgebjergskolen', 'Raarup Skole', 'Ørkildskolen', 'Thurø Skole',
       'Østofte Landsbyordning', 'Erlev Skole',
       'Fællesskolen Hammelev-Sct. Severin', 'Solbakkeskolen',
       'Stouby Skole', 'Stige Skole', 'Glyngøre Skole', 'Lumby Skole',
       'Kristrup Skole', 'Skibet Skole', 'Børneby Midt',
       'Fynshav Børneunivers', 'Tollundskolen', 'Ramløse Skole',
       'Hobrovejens Skole', 'Dagmarskolen', 'Gelsted Skole',
       'Langhøjskolen', 'Skodborg Børnecenter',
       'Fællesskolen HoptrupMarstrupVilstrup', 'Børneby Nord',
       'Koldby Skole', 'Brahesminde Skole', 'Vemb Skole og Bør